In [117]:
import pandas as pd 
import os
import warnings 
import numpy as np
warnings.filterwarnings('ignore')

_CSV_PATH = './data/csv'
_PARQUET_PATH = './data'

# Concateno csv de producción en un archivo parquet. 
# Esto debería correrse cada vez que la carpeta de data/csv se actualiza
data_files = os.listdir(_CSV_PATH)
df = pd.concat((pd.read_csv(_CSV_PATH + '/' + filename) for filename in data_files))

# Creo columnas
df['fecha_data'] = pd.to_datetime(df['fecha_data'],yearfirst=True)
df['dias'] = df['fecha_data'].dt.day
df['Th bbl/d'] = df['prod_pet'] / df['dias'] * 6.2898 / 1000 
df['Mm3/d'] = df['prod_gas'] / df['dias'] / 1000
df['Th boe/d'] = df['Th bbl/d'] + df['Mm3/d'] * 6.29

# Renombro columnas
df.rename(columns={'fecha_data':'Date'},inplace=True)

# Resuelvo datos NaN y errores
df['sub_tipo_recurso'].fillna('CONVENCIONAL',inplace=True)
df['idempresa'] = df['idempresa'].astype(str)

# Encuentro primer mes para curvas de producción
df_primer_mes = pd.DataFrame(df.groupby(by='idpozo')['Date'].min()).reset_index()
df_primer_mes.rename(columns={'Date':'primer_mes'},inplace=True)
df = df.merge(df_primer_mes,on='idpozo',how='left')
df['anio_inicio'] = df['primer_mes'].apply(lambda x: x.year)
df['mes_produccion'] = ((df['Date'] - df['primer_mes'])/np.timedelta64(1,'D')/30).round() + 1 

In [118]:
df_curve = df.groupby(by=['anio_inicio','mes_produccion','sub_tipo_recurso','tipopozo','cuenca','areapermisoconcesion','empresa']).aggregate({'Th boe/d':'sum','sigla':'count'}).reset_index()

In [122]:
df_curve.to_clipboard()

In [123]:
source = df_curve.groupby(by=['anio_inicio','mes_produccion','tipopozo']).aggregate({'Th boe/d':'sum','sigla':'sum'})
source['boe/d/pozo'] = source['Th boe/d'] * 1000 / source['sigla']

In [124]:
source.head()

Th boe/d  sigla  boe/d/pozo
anio_inicio mes_produccion tipopozo                                        
2006        1.0            Acuífero             0.310841    277    1.122169
                           Gasífero           753.313861   3185  236.519266
                           Inyección de Agua    0.395398   6813    0.058036
                           Inyección de Gas     0.005754     81    0.071041
                           Otro tipo            1.436457  21930    0.065502